In [94]:
import pandas
import pandas as pd
from ruamel import yaml
import sys
import os
# from prefect import task, Flow
import datetime
import pandas as pd
import great_expectations as ge
from great_expectations.profile.user_configurable_profiler import (
    UserConfigurableProfiler
)

import great_expectations.exceptions as ge_exceptions
from great_expectations.core.batch import (
    BatchRequest,
    RuntimeBatchRequest,
)

abspath = os.path.dirname(os.path.normpath(os.path.abspath(os.path.dirname(''))))

sys.path.insert(0, os.path.abspath('../src'))

from process.raw import *

from process.curate import *

from process.presentation import *

driver_path = os.path.join(abspath, 'drivers', 'chromedriver.exe')

raw_download_path = os.path.normpath(os.path.join(abspath, 'data', 'raw'))

curate_download_path = os.path.normpath(os.path.join(abspath, 'data', 'curated'))

presentation_download_path = os.path.normpath(os.path.join(abspath, 'data', 'presentation'))



In [76]:

hospital_urls = get_source_urls(driver_path, abspath)
""" Curated Standard Charge Data """
## Duke
df_raw_duke = get_duke(hospital_urls, raw_download_path)

df_curated_duke = curate_duke(df_raw_duke, curate_download_path)

print('Duke Downloaded')


Duke Downloaded


In [81]:
context = ge.get_context()

datasource_config = {
    "name": "example_datasource",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "module_name": "great_expectations.execution_engine",
        "class_name": "PandasExecutionEngine",
    },
    "data_connectors": {
        "default_runtime_data_connector_name": {
            "class_name": "RuntimeDataConnector",
            "module_name": "great_expectations.datasource.data_connector",
            "batch_identifiers": ["default_identifier_name"],
        },
    },
}

context.add_datasource(**datasource_config)

context.test_yaml_config(yaml.dump(datasource_config))

batch_request = RuntimeBatchRequest(
    datasource_name="example_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="duke_raw",  # This can be anything that identifies this data_asset for you
    runtime_parameters={"batch_data": df_raw_duke},  # df is your dataframe
    batch_identifiers={"default_identifier_name": "default_identifier"}
)

# expectation_suite, validation_result = BasicDatasetProfiler().profile(new_batch)

context.get_batch(batch_request)


context.create_expectation_suite(
    expectation_suite_name="test_suite", overwrite_existing=True
)
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="test_suite"
)

context.profile_datasource(validator)


profiler = UserConfigurableProfiler(
    profile_dataset=validator,
    excluded_expectations=None,
    ignored_columns=ignored_columns,
    not_null_only=False,
    primary_or_compound_key=False,
    semantic_types_dict=None,
    table_expectations_only=False,
    value_set_threshold="MANY",
)

suite = profiler.build_suite()

validator.save_expectation_suite(discard_failed_expectations=False)

context.build_data_docs()




In [130]:

import os
import great_expectations as ge

from great_expectations.profile.basic_dataset_profiler import BasicDatasetProfiler
from great_expectations.render.renderer import ProfilingResultsPageRenderer, ExpectationSuitePageRenderer
from great_expectations.render.view import DefaultJinjaPageView

import great_expectations.exceptions as ge_exceptions
from great_expectations.core.batch import (
    Batch,
    BatchDefinition,
    BatchRequest,
    IDDict,
    RuntimeBatchRequest,
)
from great_expectations.data_context.util import (
    file_relative_path,
    instantiate_class_from_config,
)
from great_expectations.datasource.data_connector import (
    ConfiguredAssetFilesystemDataConnector,
)
from great_expectations.datasource.new_datasource import Datasource

Datasource.get_batch_list_from_batch_request
# profiling_html_filepath = '/path/into/which/to/save/results.html'

# obtain the DataContext object

batch_request = BatchRequest(
    datasource_name="example_datasource",
    data_connector_name="default_inferred_data_connector_name",
    data_asset_name="duke_raw",  # this is the name of the table you want to retrieve
)

batch_request = RuntimeBatchRequest(
    datasource_name="example_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="duke_raw",  # This can be anything that identifies this data_asset for you
    runtime_parameters={"batch_data": df_raw_duke},  # df is your dataframe
    batch_identifiers={"default_identifier_name": "default_identifier"}
)

# expectation_suite, validation_result = BasicDatasetProfiler().profile(new_batch)

context.get_batch(batch_request)


BatchKwargsError: BatchKwargs must be a BatchKwargs object or dictionary.

In [ ]:

context.create_expectation_suite(
    expectation_suite_name="test_suite", overwrite_existing=True
)
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="test_suite"
)

context.profile_datasource(validator)


profiler = UserConfigurableProfiler(
    profile_dataset=validator,
    excluded_expectations=None,
    ignored_columns=ignored_columns,
    not_null_only=False,
    primary_or_compound_key=False,
    semantic_types_dict=None,
    table_expectations_only=False,
    value_set_threshold="MANY",
)

suite = profiler.build_suite()

validator.save_expectation_suite(discard_failed_expectations=False)

context.build_data_docs()


